In this notebook, we will take our previous Elo system for tennis players and add playing surface as a parameter. There are a few ways in which surface has been taken into account.

1. (Surface-only) treat each surface as a different sport altogether, so that each player has three ratings that don't interact with one another.
2. (Weighted average) take the surface-specific ratings in item 1 above and the all-surfaces ratings developed in our previous post, then take a weighted average of them, minimizing the log-loss error.
3. (Surface-dependent K-factor) According to the surface being played on, update each player's surface-specific rating according to a different K-factor and take the win probability from the corresponding surface-specific ratings.

The first and second are implemented by Jeff Sackmann, the tennis data god, where the weighted average is the actual average. (Actually on his website, he averages the raw Elo ratings themselves, but that's fairly similar---though not identical---to what I described.) The third is the idea introduced in this post, which seems fairly natural to me and perhaps a little less ad-hoc than taking the average between surface-only and surface-agnostic ratings. So let's explain how the surface-dependent K-factor (SDKF) model works.


## SDKF model

Define $$\sigma(x) = \exp(x) / (\exp(x) + 1),$$ the logistic function. If player one (p1) has rating $x$ and player two (p2) has rating $y$, the probability that p1 wins is given by $\sigma(x-y)$. Suppose $w=1$ if p1 wins and $w=0$ if p1 loses. After the match, the ratings are updated with the rule $$x \mapsto x + (-1)^{w+1} K(n_1)\sigma((-1)^w(x-y)),\quad y \mapsto y+(-1)^w K(n_2)\sigma((-1)^w (x-y)),$$ where $K$ is a function of the number of matches played by p1 ($n_1$) and the number of matches played by p2 ($n_2$). The function $K$ is of the form $$K(n) = \frac{a}{(b + n)^c}.$$

To define surface-specific ratings, we can do the following. Let $A$ be a $3\times 3$ matrix. We map surfaces to indices: index 1 refers to clay, 2 to grass, 3 to hard. Now let $\vec{x},\vec{y}\in \mathbb{R}^3$ be the ratings of p1 and p2, respectively. Specifically, $$\vec{x} = (x_1,x_2,x_3)$$ and $x_1$ is the p1 clay rating, $x_2$ is the p1 grass rating, and so on. Define $\sigma(\vec{x}) = (\sigma(x_1),\sigma(x_2),\sigma(x_3))$. If $a_{ij}$ is the $(i,j)$ entry of $A$, then we make the following change to the update rule: $$\vec{x} \mapsto \vec{x} + (-1)^{w+1}K(n_1)A\sigma((-1)^w(\vec{x}-\vec{y})), \quad \vec{y} \mapsto \vec{y} + (-1)^w K(n_2)A\sigma((-1)^w(\vec{x}-\vec{y})).$$

The matrix $A$ consists of the speed with which to update each of the three ratings, given the surface being played on. For example, if the match is being played on grass, we intuit that the result shouldn't have a large effect on the players' clay rating, but it should have a large effect on the players' grass rating. On the other hand, if the match is being played on hard, we might think that it should have an equal effect on the players' grass and clay ratings.

Finally, let's determine the win probability and the interpretation of the matrix $A$. If $$\vec{s}=\begin{cases} \vec{e}_1 &\quad \text{ if clay} \\ \vec{e}_2 &\quad \text{ if grass} \\ \vec{e}_3 &\quad \text{ if hard} \end{cases}$$ is the vector denoting surface being played on, then the win probability of p1 is $$\sigma(\vec{x}-\vec{y})\cdot \vec{s}.$$ This indicates that **$a_{ij}$ is the update speed for the players' surface $i$ rating if the playing surface is $j$**.

### Special cases
It is instructive to examine special cases of $A$.
1. If $A$ is the identity matrix, then no surface affects any other surface, and all the update coefficients are equal. So this would be equivalent to treating each surface as a different sport altogether (Surface-only ratings).
2. If $A$ is the all-ones matrix, then all surfaces are treated equally. This results in surface-agnostic ratings, which is the classical setting.

Based on these two extremes, we expect an effective $A$ to have heavy diagonal entries but nonzero off-diagonal entries, all positive. For our $K$, we take $a,b,c$ to be $$(a,b,c) = (0.47891635, 4.0213623 , 0.25232273)$$ based on training data from 1980 to 2014, from the previous post. Then we initialize the entries of $A$ to be uniform random numbers between 0 and 1.5.

In [1]:
import numpy as np
import pandas as pd

np.random.seed(5192020) # today's date!

def sigmoid(z):
    '''The sigmoid function.'''
    return np.where(z >= 0, 
                    1 / (1 + np.exp(-z)), 
                    np.exp(z) / (1 + np.exp(z)))

class Elo_sdkf:
    
    def __init__(self, start_year, end_year, num_models, 
                 k_param = np.array([0.47891635, 4.0213623 , 0.25232273]),
                 lower = np.zeros(9), upper = np.ones(9) * 1.5):
        
        self.start_year = start_year
        self.end_year = end_year
        self.num_models = num_models
        
        self.k_params = np.multiply(np.ones((num_models, 3)), k_param)
        self.a_params = np.multiply(np.random.random((num_models, 9)), upper - lower) + lower
        
        self.data = []
        for i in range(start_year, end_year + 1):
            self.data.append(pd.read_csv('./atp/atp_matches_' + str(i) + '.csv'))
        
        # collect all the player names
        self.players = {player for player in self.data[0]['winner_name']}
        self.players = self.players.union({player for player in self.data[0]['loser_name']})
        for i in range(1, end_year - start_year + 1):
            self.players = self.players.union({player for player in self.data[i]['winner_name']})
            self.players = self.players.union({player for player in self.data[i]['loser_name']})            
        
        # ratings are of the form (n, r)
        # where n is the number of matches the player has played
        # and r is their rating.
        self.ratings = {player: (0, np.ones((num_models,3))) for player in self.players}
        self.select_ratings = {player: (0,1.0) for player in self.players}
        
        col_names = ['a'+str(i) for i in range(1,10)]
        self.params_data = pd.DataFrame({a : self.a_params[:,i] for (a,i) in zip(col_names, range(9))})
        self.params_data['k1'] = self.k_params[:,0]
        self.params_data['k2'] = self.k_params[:,1]
        self.params_data['k3'] = self.k_params[:,2]
        self.params_data['ll'] = np.zeros(num_models)
        self.params_data['wp'] = np.zeros(num_models)
        self.params_data['bs'] = np.zeros(num_models)
        
    def get_params(self):
        return self.k_params.copy(), self.a_params.copy()
    
    def set_params(self, k_ps, a_ps):
        self.k_params, self.a_params = k_ps, a_ps
        
    def get_params_data(self):
        return self.params_data.copy()
    
    def k(self, n, ps):
        '''returns the vector K-factor, which dictates how sensitive ratings are
        to an individual match and depends on the number of matches played.'''
        return np.multiply(ps[:,0], 
                           np.power(ps[:,1] + n, -ps[:,2])
                          )
    
    
    def update_one(self, x, y, n1, n2, k_params, a_params, s):
        '''this function updates one match. 'x','y' are the ratings of
        the winner and loser respectively, 'n1','n2' are the number of matches
        that the winner and loser have played respectively. Returns the
        prior probability that the winner wins, and the values to update 
        the ratings by. '''
        z = np.multiply(np.dot(a_params.reshape((len(a_params),3,3)), s), sigmoid(y-x))
        z1 = z[:,0]
        z2 = z[:,1]
        z3 = z[:,2]
        u1 = np.multiply(self.k(n1, k_params), z1)
        u2 = np.multiply(self.k(n1, k_params), z2)
        u3 = np.multiply(self.k(n1, k_params), z3)
        v1 = -np.multiply(self.k(n2, k_params), z1)
        v2 = -np.multiply(self.k(n2, k_params), z2)
        v3 = -np.multiply(self.k(n2, k_params), z3)
        
        u = np.transpose(np.array([u1,u2,u3]))
        v = np.transpose(np.array([v1,v2,v3]))
        prob = np.dot(sigmoid(x-y), s)
        return(prob, u, v)
    
    def update_all_ratings(self, year):
        '''update all the ratings at once. '''
        # first reset the ratings.
        self.ratings = {player: (0, np.ones((self.num_models,3))) 
                            for player in self.players}
        
        ll = np.zeros(self.num_models)
        wp = np.zeros(self.num_models)
        bs = np.zeros(self.num_models)
        
        for i in range(len(self.data)):
            for j, row in self.data[i].iterrows():
                
                winner = row['winner_name']
                loser = row['loser_name']
                surface = row['surface']
                if surface == 'Clay':
                    s = np.array([1,0,0])
                elif surface == 'Hard':
                    s = np.array([0,0,1])
                else: # Carpet gets classified as Grass.
                    s = np.array([0,1,0])
                
                # get ratings.
                wnm, wrating = self.ratings[winner]
                lnm, lrating = self.ratings[loser]
                
                # update.
                prob_vec, u1, u2 = self.update_one(wrating, lrating, wnm, lnm, 
                                                   self.k_params, self.a_params, s)
                self.ratings[winner] = wnm + 1, wrating + u1
                self.ratings[loser] = lnm + 1, lrating + u2
                
                # compute log-loss error or win prediction percentage.
                if(i + self.start_year >= year):
                    ll -= np.log(prob_vec)
                    wp += (prob_vec > 0.5).astype(float)
                    bs += np.power(1-prob_vec, 2)
        
        # figure out what to divide cost by.
        num_rows = 0
        for i in range(min(len(self.data), self.end_year + 1 - year)):
            num_rows += len(self.data[year - self.start_year - 1 + i])

        self.params_data['ll'] = ll / num_rows
        self.params_data['wp'] = wp / num_rows
        self.params_data['bs'] = bs / num_rows
    
    def sort_params(self, method = 'll'):
        '''sort the parameters according to cost method.'''
        if method == 'll':
            self.params_data = self.params_data.sort_values(by='ll')
        elif method == 'wp':
            self.params_data = self.params_data.sort_values(by='wp', ascending = False)
        else:
            self.params_data = self.params_data.sort_values(by='bs')
    
    def update_select_ratings(self, year, a_params):
        '''this function updates only the ratings corresponding to the given
        parameters. It takes the average probability.'''
        
        # first reset the ratings.
        n = len(self.k_params)
        self.select_ratings = {player: (0, np.ones((n,3))) for player in self.players}
        
        ll = 0
        wp = 0
        bs = 0
        
        for i in range(len(self.data)):
            for j, row in self.data[i].iterrows():
                
                winner = row['winner_name']
                loser = row['loser_name']
                surface = row['surface']
                if surface == 'Clay':
                    s = np.array([1,0,0])
                elif surface == 'Hard':
                    s = np.array([0,0,1])
                else: # Carpet gets classified as Grass.
                    s = np.array([0,1,0])
                
                
                # get ratings.
                wnm, wrating = self.select_ratings[winner]
                lnm, lrating = self.select_ratings[loser]
                
                # update.
                prob, u1, u2 = self.update_one(wrating, lrating, wnm, lnm, 
                                               self.k_params, a_params, s)
                self.select_ratings[winner] = wnm + 1, wrating + u1
                self.select_ratings[loser] = lnm + 1, lrating + u2
                
                # what's the prob?
                prob = np.mean(prob)
                
                # compute log-loss error or win prediction percentage.
                if(i + self.start_year >= year):
                    ll -= np.log(prob)
                    wp += (prob > 0.5).astype(float)
                    bs += (1-prob)**2
        
        # figure out what to divide cost by.
        num_rows = 0
        for i in range(min(len(self.data), self.end_year + 1 - year)):
            num_rows += len(self.data[year - self.start_year - 1 + i])

        return (ll / num_rows, wp / num_rows, bs / num_rows)
        

In [2]:
elo_sdkf = Elo_sdkf(2000, 2013, 10000)
elo_sdkf.update_all_ratings(2010)
elo_sdkf.sort_params()
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())

            ll        wp        bs
6582  0.575501  0.678254  0.197409
8772  0.575569  0.679325  0.197355
4473  0.575620  0.679489  0.197426
3921  0.575660  0.679572  0.197463
10    0.575678  0.679489  0.197462


In [3]:
col_names = ['a'+str(i) for i in range(1,10)]
print(df[col_names].head())

            a1        a2        a3        a4        a5        a6        a7  \
6582  1.497282  0.709504  0.223734  0.035879  1.016345  0.891814  0.635505   
8772  1.496927  0.122563  0.659796  0.035299  0.704547  1.405535  0.895188   
4473  1.403872  0.472810  0.730792  0.233342  0.614701  1.108361  0.540628   
3921  1.459754  0.449260  0.745738  0.556220  1.398105  0.590583  0.627694   
10    1.459020  0.305191  0.589823  0.331715  1.321166  0.879316  0.332806   

            a8        a9  
6582  1.089314  1.143836  
8772  0.967678  1.212636  
4473  1.486120  1.129831  
3921  0.869944  1.246126  
10    0.439226  1.318882  


Since there are so many parameters now, uniform distribution gets sparser. Let's tighten up our range by taking the top 50 parameters and setting the uniform distribution to be around their means.

In [4]:
print(np.mean(np.array(df[col_names].iloc[:50]), axis=0))
print(np.var(np.array(df[col_names].iloc[:50]), axis=0))

[1.30804042 0.42005384 0.57899851 0.27871053 0.94740469 0.88936044
 0.58987385 0.93457748 1.20006381]
[0.01910083 0.11820104 0.05413589 0.04091668 0.1067347  0.10136307
 0.04194059 0.09898853 0.02578975]


In [5]:
lower = np.mean(np.array(df[col_names].iloc[:50]), axis=0) - 4*np.var(np.array(df[col_names].iloc[:50]), axis=0)
upper = np.mean(np.array(df[col_names].iloc[:50]), axis=0) + 4*np.var(np.array(df[col_names].iloc[:50]), axis=0)

In [6]:
elo_sdkf = Elo_sdkf(1995, 2013, 10000, lower=lower, upper=upper)
elo_sdkf.update_all_ratings(2010)

In [7]:
elo_sdkf.sort_params()
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())
elo_sdkf.sort_params(method = 'wp')
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())
elo_sdkf.sort_params(method = 'bs')
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())

            ll        wp        bs
3746  0.574072  0.679654  0.196739
6182  0.574075  0.680478  0.196746
1213  0.574086  0.680313  0.196760
425   0.574098  0.680560  0.196769
584   0.574109  0.681054  0.196764
            ll        wp        bs
624   0.574792  0.682867  0.197084
1489  0.575063  0.682784  0.197146
3537  0.574289  0.682702  0.196845
3341  0.574374  0.682702  0.196876
7902  0.574644  0.682619  0.196957
            ll        wp        bs
3746  0.574072  0.679654  0.196739
6182  0.574075  0.680478  0.196746
4921  0.574123  0.678336  0.196752
1213  0.574086  0.680313  0.196760
584   0.574109  0.681054  0.196764


In [8]:
col_names = ['a'+str(i) for i in range(1,10)]
print(df[col_names].head())

            a1        a2        a3        a4        a5        a6        a7  \
3746  1.383559  0.186148  0.582325  0.163048  1.366782  0.642451  0.640518   
6182  1.367565 -0.038163  0.501433  0.141208  1.225584  0.738717  0.557209   
4921  1.367434  0.034490  0.492734  0.135281  0.840048  0.683265  0.598902   
1213  1.347424  0.120422  0.612078  0.116368  1.020613  0.703088  0.524921   
584   1.373804  0.209944  0.514236  0.163280  1.325427  0.585647  0.450906   

            a8        a9  
3746  1.258164  1.285806  
6182  0.815735  1.200875  
4921  1.250441  1.270224  
1213  1.242716  1.166514  
584   0.925887  1.271030  


In [9]:
a_params = np.array(df[col_names].iloc[:50])
elo_sdkf = Elo_sdkf(1980, 2014, 1)
elo_sdkf.update_select_ratings(2014, a_params = a_params)

(0.5792334709883984, 0.673233695652174, 0.19887088255743884)

In [10]:
a_params

array([[ 1.38355895,  0.18614784,  0.58232495,  0.16304785,  1.36678152,
         0.6424513 ,  0.64051841,  1.25816376,  1.28580581],
       [ 1.3675646 , -0.03816339,  0.50143259,  0.14120843,  1.22558376,
         0.73871681,  0.5572088 ,  0.81573485,  1.20087467],
       [ 1.3674336 ,  0.03449045,  0.49273421,  0.13528091,  0.84004762,
         0.68326467,  0.59890153,  1.25044115,  1.2702242 ],
       [ 1.34742374,  0.12042227,  0.61207821,  0.11636838,  1.02061274,
         0.70308823,  0.52492127,  1.24271577,  1.16651379],
       [ 1.37380444,  0.20994372,  0.51423607,  0.16327988,  1.32542685,
         0.58564659,  0.45090589,  0.92588663,  1.27102954],
       [ 1.34748103,  0.10105228,  0.44034158,  0.1387066 ,  0.99520008,
         0.67828855,  0.51300676,  1.21462154,  1.26050307],
       [ 1.33221687, -0.04874868,  0.55306821,  0.12238943,  1.31506376,
         0.59028718,  0.43069407,  0.83557575,  1.18334558],
       [ 1.34757652, -0.04455222,  0.46752248,  0.13306174,  1

## Testing on 2015-2019 data
Recall the 2015-2019 comparison between the randomly initialized Elo model and the FiveThirtyEight Elo model:
```
  optimized_for        ll        wp        bs
0            ll  0.607112  0.658860  0.209942
1            wp  0.623405  0.657555  0.214121
2            bs  0.607411  0.658723  0.209912
3           538  0.611903  0.661607  0.210791
```

Let's try our SDKF model.

In [11]:
elo_sdkf = Elo_sdkf(1980, 2019, 1)
elo_sdkf.update_select_ratings(2015, a_params = a_params)

(0.6031931424789028, 0.663804945054945, 0.20829817411371923)

So we made a pretty significant improvement in log-loss error over FiveThirtyEight's model, while eliminating hand-picking of hyperparameters as much as possible. 

## Comparison with SA+SO average: BUMMER
Now let's build a model in which the win probability is averaged between the predictions given by the surface-agnostic rating and the surface-only rating. With our framework, the respective $A$s are $$ \begin{pmatrix} 1&1&1\\1&1&1\\1&1&1\end{pmatrix}, \quad \begin{pmatrix}1&0&0\\0&1&0\\0&0&1\end{pmatrix}.$$ The average method fits right into our framework and can be computed with one line of code.

In [12]:
elo_avg = Elo_sdkf(1980,2019,1)
elo_avg.update_select_ratings(2015, a_params = np.array([
    np.ones(9),
    np.array([1,0,0,0,1,0,0,0,1])
]))

(0.6026983365860035, 0.6609203296703297, 0.20832241189833223)

Shoot, this is as good as mine, and a lot simpler...

## Conclusion
Well, this is a bit of a bummer for me since this model almost identically to mine, and its parameters are a lot simpler. But this model still uses my hyperparameter principles from the previous notebook, and it fits quite nicely into the mathematical framework I've made here. If I could search for parameters like a thousand times faster, I might try to optimize for a weighted average of probabilities from two matrices, to get results like the above. With our current parameter search, we can't find this local min because neither matrix by itself does well, but averaged together they do well. Doing this larger parameter search, we'd need to search through, in all probably, ten million parameters or so... which doesn't seem all that bad, in the context of all the deep learning stuff I'm doing recently. So next up, I'm going to load this up in Google Colab and see if I can run it.